In [1]:
!pip install deep_sort_realtime

   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB 187.9 kB/s eta 0:00:45
   ---------------------------------------- 0.0/8.4 MB 163.4 kB/s eta 0:00:52
   ---------------------------------------- 0.1/8.4 MB 272.3 kB/s eta 0:00:31
   ---------------------------------------- 0.1/8.4 MB 255.1 kB/s eta 0:00:33
   ---------------------------------------- 0.1/8.4 MB 255.1 kB/s eta 0:00:33
   ---------------------------------------- 0.1/8.4 MB 255.1 kB/s eta 0:00:33
    --------------------------------------- 0.1/8.4 MB 273.1 kB/s eta 0:00:31
    --------------------------------------- 0.1/8.4 MB 273.1 kB/s eta 0:00:31
    --------------------------------------- 0.1/8.4 MB 283.8 kB/s eta 0:00:30
    ------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\divu2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

In [3]:
# load YOLOv8 model
yolo_model=YOLO("yolov8n.pt")  # Load the YOLOv8 model (you can change to yolov8s.pt, yolov8m.pt, etc.)

In [4]:
# Initialize DeepSORT Tracker
tracker=DeepSort(max_age=30)
# max_age=30 after 30 frames if object  disappears then tracker will remove it from it from the object list

In [ ]:
# open webcam
cap=cv2.VideoCapture('car.mp4')
#Reads continuous video frames
while cap.isOpened(): 
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run YOLOv8 detection
    results = yolo_model(frame)
    detections = [] 
    
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            detections.append([x1, y1, x2, y2, conf])
    
    # Convert to numpy array
    detections = np.array(detections) 
    
    # Update tracker
    tracked_objects = tracker.update(detections)
    
    # Draw tracking boxes
    for obj in tracked_objects:
        x1, y1, x2, y2, track_id = obj.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display output
    cv2.imshow("Object Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"): 
        break

cap.release()
cv2.destroyAllWindows()